# United States - Crime Rates - 1960 - 2014

### Introduction:

This time you will create a data 

Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window

In [2]:
spark = SparkSession.builder \
    .appName('Spark exercises based on pandas exercises') \
    .getOrCreate()

spark

23/03/03 20:41:43 WARN Utils: Your hostname, karlos-300E5M-300E5L resolves to a loopback address: 127.0.1.1; using 192.168.10.20 instead (on interface enp1s0)
23/03/03 20:41:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 20:41:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/US_Crime_Rates/US_Crime_Rates_1960_2014.csv). 

### Step 3. Assign it to a variable called crime.

In [23]:
FILEPATH = 'https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/US_Crime_Rates/US_Crime_Rates_1960_2014.csv'

crime = spark.read.csv(
    './US_Crime_Rates_1960_2014.csv',
    sep=',',
    header=True,
    inferSchema=True
)

crime.show()

+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|Year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+----+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|1960| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
|1961| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
|1962| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      2089600|       366800|
|1963| 188483000| 4109500| 316970| 3792500|  8640|        17650| 116470|            174210| 1086400|      2297800|       408300|
|1964| 191141000| 4564600| 364220| 4200400|  9360|        21420| 130390|            203050| 12132

### Step 4. What is the type of the columns?

In [24]:
crime.dtypes

[('Year', 'int'),
 ('Population', 'int'),
 ('Total', 'int'),
 ('Violent', 'int'),
 ('Property', 'int'),
 ('Murder', 'int'),
 ('Forcible_Rape', 'int'),
 ('Robbery', 'int'),
 ('Aggravated_assault', 'int'),
 ('Burglary', 'int'),
 ('Larceny_Theft', 'int'),
 ('Vehicle_Theft', 'int')]

##### Have you noticed that the type of Year is int64. But pandas has a different type to work with Time Series. Let's see it now.

### Step 5. Convert the type of the column Year to datetime64

In [25]:
crime = crime.withColumn(
    'year',
    F.to_date('year', 'y')
)

print(crime)
crime.show()

DataFrame[year: date, Population: int, Total: int, Violent: int, Property: int, Murder: int, Forcible_Rape: int, Robbery: int, Aggravated_assault: int, Burglary: int, Larceny_Theft: int, Vehicle_Theft: int]
+----------+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|      year|Population|   Total|Violent|Property|Murder|Forcible_Rape|Robbery|Aggravated_assault|Burglary|Larceny_Theft|Vehicle_Theft|
+----------+----------+--------+-------+--------+------+-------------+-------+------------------+--------+-------------+-------------+
|1960-01-01| 179323175| 3384200| 288460| 3095700|  9110|        17190| 107840|            154320|  912100|      1855400|       328200|
|1961-01-01| 182992000| 3488000| 289390| 3198600|  8740|        17220| 106670|            156760|  949600|      1913000|       336000|
|1962-01-01| 185771000| 3752200| 301510| 3450700|  8530|        17550| 110860|            164570|  994300|      208960

### Step 6. Set the Year column as the index of the dataframe

In [26]:
# It does not make sense in Spark data model.

### Step 7. Delete the Total column

In [27]:
crime.drop('Total')

DataFrame[year: date, Population: int, Violent: int, Property: int, Murder: int, Forcible_Rape: int, Robbery: int, Aggravated_assault: int, Burglary: int, Larceny_Theft: int, Vehicle_Theft: int]

### Step 8. Group the year by decades and sum the values

#### Pay attention to the Population column number, summing this column is a mistake

In [70]:
crime_aggregate_by_decade = crime \
    .withColumn(
        'Decade',
        F.floor(F.year('Year') / 10) * 10
    ).groupBy('Decade') \
    .agg(
        {column: 'sum' for column in crime.columns if column not in ('year', 'Population')}
    )

crime_aggregate_by_decade


DataFrame[Decade: bigint, sum(Larceny_Theft): bigint, sum(Aggravated_assault): bigint, sum(Burglary): bigint, sum(Total): bigint, sum(Vehicle_Theft): bigint, sum(Murder): bigint, sum(Robbery): bigint, sum(Violent): bigint, sum(Property): bigint, sum(Forcible_Rape): bigint]

In [71]:
crime_aggregate_by_decade.select(
        'Decade',
        'sum(Total)',
        'sum(Murder)',
        'sum(Violent)',
        'sum(Robbery)'
    ).orderBy('Decade') \
    .show()

+------+----------+-----------+------------+------------+
|Decade|sum(Total)|sum(Murder)|sum(Violent)|sum(Robbery)|
+------+----------+-----------+------------+------------+
|  1960|  49295900|     106180|     4134930|     1633510|
|  1970| 100991600|     192230|     9607930|     4159020|
|  1980| 131123369|     206439|    14074328|     5383109|
|  1990| 136582146|     211664|    17527048|     5748930|
|  2000| 115012044|     163068|    13968056|     4230366|
|  2010|  50167967|      72867|     6072017|     1749809|
+------+----------+-----------+------------+------------+



### Step 9. What is the most dangerous decade to live in the US?

In [72]:
# According to Total:

crime_aggregate_by_decade.select('Decade', 'sum(total)') \
    .orderBy('sum(total)', ascending=False) \
    .show()

+------+----------+
|Decade|sum(total)|
+------+----------+
|  1990| 136582146|
|  1980| 131123369|
|  2000| 115012044|
|  1970| 100991600|
|  2010|  50167967|
|  1960|  49295900|
+------+----------+

